# Installation and Repository cloning

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [2]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


## Load and Prepare Adversarial Model

In [3]:
model = 'Wang2023Better_WRN-28-10' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10
net, art_net, fb_net, alias = utils.get_model(modelname=model, norm='L2') # specify the norm in case of robustbench models

/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/robustbench/utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_lo

## Load the dataset

In [4]:
splitsize = 1000 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset_split=splitsize)

Files already downloaded and verified


## Test Accuracy

In [5]:
utils.test_accuracy(net, xtest, ytest)


Accuracy of the test set is: 94.000%



## Parameters 

In [9]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.1,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations =100, 
                        verbose=False)

# Hyperparameter Sweep Experiment

In [ ]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack_l1'
hyperparameter_range=[1]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

In [10]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'deep_fool',
                #'brendel_bethge',
                #'carlini_wagner_l2',
                #'elastic_net',
                #'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1_l2',
                #'auto_projected_gradient_descent',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                'exp_attack_l1',
                'original_AutoAttack_apgd-only'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)

		-------------------------- Processing Attack: exp_attack_l1 --------------------------

20 images done. Current Adversarial Accuracy (L1 / L2/ EN): 35.0 / 95.0/35.0%
40 images done. Current Adversarial Accuracy (L1 / L2): 45.0 / 90.0%
60 images done. Current Adversarial Accuracy (L1 / L2/ EN): 50.0 / 91.66666666666667/50.0%
80 images done. Current Adversarial Accuracy (L1 / L2/ EN): 52.5 / 92.5/52.5%
100 images done. Current Adversarial Accuracy (L1 / L2/ EN): 57.0 / 94.0/57.0%

Adversarial accuracy (L1 / L2/ EN): 57.0000 / 94.0000/ 57.0000%


average sparsity: 77.9957%


Total runtime:  109.2887 seconds

attack success rate in epsilon (L1 / L2):  39.3617  /  0.0
mean adv. distance (L1 / L2):  11.9931  /  1.5479
		-------------------------- Processing Attack: original_AutoAttack_apgd-only --------------------------

20 images done. Current Adversarial Accuracy (L1 / L2/ EN): 35.0 / 95.0/35.0%
40 images done. Current Adversarial Accuracy (L1 / L2): 47.5 / 90.0%
60 images done. Current